In [1]:
# Fix randomness and hide warnings
seed = 42

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['TF_USE_LEGACY_KERAS'] = '1'
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'
os.environ["KERAS_BACKEND"] = "tensorflow"

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import random
random.seed(seed)

import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight

from tensorflow.keras import mixed_precision

print(tf.__version__)

# Importa il modello FasterViT
from keras_cv_attention_models import fastervit
mixed_precision.set_global_policy('mixed_float16')

E0000 00:00:1731946209.677853  502208 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731946209.699161  502208 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


2.18.0
INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4070 Ti SUPER, compute capability 8.9


In [ ]:
data_dir = "MO2/classes"
batch_size = 64
final_image_size = (309, 298)

# Carica tutte le immagini dalle directory "classes" e "augmented"
train_ds = tfk.preprocessing.image_dataset_from_directory(
    data_dir,
    image_size=final_image_size,
    batch_size=batch_size,
    shuffle=True,
    label_mode='int',
    seed=seed)

data_dir = "MO2/augmented_data"

# Carica tutte le immagini dalle directory "classes" e "augmented"
train_ds2 = tfk.preprocessing.image_dataset_from_directory(
    data_dir,
    image_size=final_image_size,
    batch_size=batch_size,
    shuffle=True,
    label_mode='int',
    seed=seed)

In [ ]:
data_dir = "MO2/classes"
batch_size = 64
final_image_size = (309, 298)

# Carica tutte le immagini dalle directory "classes" e "augmented"
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    image_size=final_image_size,
    batch_size=batch_size,
    shuffle=True,
    label_mode='int',
    seed=seed)

In [ ]:

np.save('classes.npy', train_ds.class_names)

In [ ]:
# Combina i due dataset
combined_dataset = train_ds.concatenate(train_ds2)
del train_ds, train_ds2

# Ottieni le etichette delle classi dal dataset
#labels = np.concatenate([y for x, y in combined_dataset], axis=0)

In [ ]:
combined_dataset.class_names

In [ ]:
# Combina i due dataset
combined_dataset = train_ds.concatenate(train_ds2)
del train_ds, train_ds2

# Ottieni le etichette delle classi dal dataset
labels = np.concatenate([y for x, y in combined_dataset], axis=0)

# Calcola i pesi delle classi
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(labels),
    y=labels
)

# Crea un dizionario per i pesi delle classi
class_weight_dict = dict(enumerate(class_weights))

In [ ]:
# Salva il dizionario in un file JSON
import json
with open('class_weight_dict.json', 'w') as f:
    json.dump(class_weight_dict, f)

print("Il dizionario dei pesi delle classi è stato salvato in 'class_weight_dict.json'.")

In [ ]:
plt.figure(figsize=(10, 10))
for images, lab in combined_dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(labels[lab[i]])
    plt.axis("off")

In [ ]:
# Definisci la dimensione del dataset
dataset_size = sum(1 for _ in combined_dataset)

# Definisci la proporzione per il training set
train_size = int(0.8 * dataset_size)

# Usa il metodo take per ottenere il training set
train_dataset = combined_dataset.take(train_size)

# Usa il metodo skip per ottenere il validation set
validation_dataset = combined_dataset.skip(train_size)

# Verifica il numero di immagini nei due dataset
num_train_images = sum(1 for _ in train_dataset)
num_validation_images = sum(1 for _ in validation_dataset)
print(f"Numero di immagini nel training set: {num_train_images}")
print(f"Numero di immagini nel validation set: {num_validation_images}")
del combined_dataset

In [ ]:
%load_ext jupyter_ai_magics

In [ ]:
%%ai chatgpt

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
validation_dataset = validation_dataset.map(lambda x, y: (normalization_layer(x), y))

In [ ]:
from IPython.display import clear_output
from tensorflow.keras.callbacks import Callback

class PlotLearning(Callback):
    """
    Callback for plotting the learning curves of a model during training.

    This callback records and visualizes training and validation metrics (e.g.,
    loss, accuracy) after each epoch to provide insights into the model's
    performance.

    Methods:
    - on_train_begin: Initializes a dictionary to store training metrics.
    - on_epoch_end: Collects and stores metrics after each epoch and plots the
    learning curves.
    """
    def on_train_begin(self, logs={}):
        # Initializes a dictionary to store training metrics
        self.metrics = {}
        for metric in logs:
            self.metrics[metric] = []


    def on_epoch_end(self, epoch, logs={}):
        # Storing metrics
        for metric in logs:
            if metric in self.metrics:
                self.metrics[metric].append(logs.get(metric))
            else:
                self.metrics[metric] = [logs.get(metric)]

        # Plotting
        metrics = [x for x in logs if 'val' not in x]

        # Set up subplots
        f, axs = plt.subplots(1, len(metrics), figsize=(15,5))
        clear_output(wait=True)

        # Convert axs to a numpy array for indexing
        if not isinstance(axs, np.ndarray):
            axs = np.array([axs])

        # Plot learning curves
        for i, metric in enumerate(metrics):
            axs[i].plot(range(1, epoch + 2),
                        self.metrics[metric],
                        label=metric)
            if logs['val_' + metric]:
                axs[i].plot(range(1, epoch + 2),
                            self.metrics['val_' + metric],
                            label='val_' + metric)

            axs[i].legend()
            axs[i].grid()

        plt.tight_layout()
        plt.show()

In [ ]:
# Define the path pattern for saving checkpoints and extract the directory
checkpoint_path = "checkpoints/cp-{epoch:04d}.keras"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Configure a callback to save the best model based on validation accuracy
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=False)

In [ ]:
# Carica il modello pre-addestrato
mm = fastervit.FasterViT0(pretrained="imagenet", input_shape=(309, 298, 3))
# Rimuovi l'ultimo livello del modello
mm = tfk.Model(inputs=mm.input, outputs=mm.layers[-3].output)
mm.trainable = False
def build_fastervitv0():
    """
    Build a FasterViT-based CNN with image augmentation, and a custom
    classifier for multi-class classification.
    """
    tf.random.set_seed(seed)
    inputs = tf.keras.Input(shape=(309, 298, 3))

    # Data Augmentation
    data_augmentation = tf.keras.Sequential([
        tfkl.RandomFlip("horizontal_and_vertical"),
    ])
    x = data_augmentation(inputs)

    # Passa l'input attraverso il modello pre-addestrato senza la testa di classificazione
    x1 = mm(x)
    
    # Applica GlobalAveragePooling2D per ridurre le dimensioni spaziali
    x2 = tfkl.GlobalAveragePooling2D(name="avg_pool")(x1)
    norm = tfkl.BatchNormalization(name="batch_normalization")(x2)

    # Aggiungi un nuovo classificatore in cima al modello pre-addestrato
    intermediate1 = tfkl.Dense(4096, activation=tf.keras.activations.swish)(norm)
    batch = tfkl.BatchNormalization(name="batch_normalization_1")(intermediate1)
    drop1 = tfkl.Dropout(0.25)(batch)
    intermediate2 = tfkl.Dense(2048, activation=tf.keras.activations.swish)(drop1)
    batch = tfkl.BatchNormalization(name="batch_normalization_2")(intermediate2)
    outputs = tfkl.Dense(1800, activation='softmax', dtype='float32')(batch)

    # Crea un modello collegando input e output
    model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

    # Learning Rate Scheduler
    def scheduler(epoch, lr):
        if epoch < 10:
            return lr
        else:
            return lr * tf.math.exp(-0.1)

    lr_scheduler = tfk.callbacks.LearningRateScheduler(scheduler)

    # Compila il modello con Sparse Categorical Cross-Entropy loss e Lion optimizer
    model.compile(loss=tfk.losses.SparseCategoricalCrossentropy(from_logits=True),
                  optimizer=tfk.optimizers.Lion(),
                  metrics=[
                      tfk.metrics.SparseCategoricalAccuracy(name="accuracy"),
                      tfk.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
                  ])
    model.summary()

    # Ritorna il modello e il learning rate scheduler
    return model, lr_scheduler

# Costruisci il modello
tl_model, lr_scheduler = build_fastervitv0()

In [ ]:
# Configura il callback per TensorBoard
tensorboard_callback = tfk.callbacks.TensorBoard(log_dir='./logs')

# Configura il callback per ridurre il learning rate on plateau
reduce_lr = tfk.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=5, min_lr=0.0001)

# Configura il callback per EarlyStopping
early_stopping = tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True)

csv_logger = tfk.callbacks.CSVLogger('training.log', separator=",", append=True)


In [ ]:
#tl_model.load_weights("new_model3/cp-0002.keras")
#tl_model = tfk.models.load_model("new_model4/cp-0004.keras")
#tl_model.summary()

In [ ]:
# Train the model with transfer learning
tl_history = tl_model.fit(
    train_dataset, # input training data
    validation_data=validation_dataset,
    epochs = 50,
    initial_epoch=25,
    class_weight=class_weight_dict,
    callbacks = [early_stopping, lr_scheduler, tensorboard_callback, reduce_lr, csv_logger, cp_callback]
).history

In [ ]:
tl_model.save("tl_model.keras")

In [ ]:
# Save weights of the Transfer Learning model
#tl_model.save_weights('new.weights.h5')
# Create a new instance of CustomModel for fine-tuning
#ft_model = build_effnetv2()
ft_model = tfk.models.load_model("new_FT2/cp-0001.keras")
# Load the weights for the model
#ft_model.load_weights('new.weights.h5')

# Set all layers as trainable
ft_model.trainable = True

if all(layer.trainable for layer in ft_model.layers):
    print("Successful!")
else:
    print("Error!")

ft_model.get_layer('batch_normalization').trainable = False
ft_model.get_layer('batch_normalization_2').trainable = False

In [ ]:
tl_model.get_layer('batch_normalization').trainable = False
tl_model.get_layer('batch_normalization_1').trainable = False
tl_model.get_layer('batch_normalization_2').trainable = False
tl_model.get_layer('dense_3').trainable = False
tl_model.get_layer('dropout_1').trainable = False
tl_model.get_layer('dense_4').trainable = False
tl_model.get_layer('dense_5').trainable = False
tl_model.get_layer('avg_pool').trainable = False

In [ ]:
# Display the summary of the Transfer Learning model
tl_model.summary()

# Display the layers and their trainable status within the 'model_1' submodule
print("Layers within 'FasterViT0' submodule:")
for i, layer in enumerate(tl_model.get_layer('model_1').layers): # To adjust
    print(i, layer.name, layer.trainable)

print("\n")

# Display all layers in the entire model and their trainable status
print("All layers in the model:")
for i, layer in enumerate(tl_model.layers):
    print(i, layer.name, layer.trainable)

In [ ]:
# Run this cell to run half of the layers.

# Freeze the first N layers (approximately the first half) of the
# 'convnext_large' submodule.
# Also the fully-connected part will train.
N = 535 # To adjust
for i, layer in enumerate(tl_model.get_layer('model_1').layers[:N]):# To adjust
    layer.trainable = True

# Display the layers and their updated trainable status within the submodule
print("Updated trainable status after freezing layers:")
for i, layer in enumerate(tl_model.get_layer('model_1').layers):# To adjust
    print(i, layer.name, layer.trainable)

In [ ]:
# Define the path pattern for saving checkpoints and extract the directory
checkpoint_path = "FT2/cp-{epoch:04d}.keras"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Configure a callback to save the best model based on validation accuracy
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=False)

In [ ]:
# Configura il callback per TensorBoard
tensorboard_callback = tfk.callbacks.TensorBoard(log_dir='./logs_FT')

# Configura il callback per ridurre il learning rate on plateau
reduce_lr = tfk.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=2, min_lr=1e-6)

csv_logger = tfk.callbacks.CSVLogger('training_FT.log', separator=",", append=True)

In [ ]:
# Scongela tutti i layer del modello pre-addestrato (FasterViT0)
for layer in tl_model.get_layer('model_1').layers:
    layer

In [ ]:
tl_model.get_layer('model_1').trainable = True

In [ ]:
# Verifica che i layer siano effettivamente scongelati
for layer in tl_model.layers:
    print(layer.name, layer.trainable)

In [ ]:
# Compila di nuovo il modello con un learning rate basso
tl_model.compile(optimizer=tfk.optimizers.Lion(learning_rate=2.5e-6),
                 loss=tfk.losses.SparseCategoricalCrossentropy(from_logits=True),
                 metrics=[
                     tfk.metrics.SparseCategoricalAccuracy(name="accuracy"),
                     tfk.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
                 ])

In [ ]:
tl_model.summary()

In [ ]:
# Train the model with transfer learning
tl_history = tl_model.fit(
    train_dataset, # input training data
    validation_data=validation_dataset,
    epochs = 5,
    class_weight=class_weight_dict,
    callbacks = [tensorboard_callback, reduce_lr, csv_logger, cp_callback]
).history

In [3]:
model = tfk.models.load_model("FT2/cp-0005.keras")

In [4]:
tf.saved_model.save(model, os.path.join(os.getcwd(), "fastervit/"))

INFO:tensorflow:Assets written to: /home/penzo/workspace/MushDex/YOLO11/detection_training_final/fastervit/assets


INFO:tensorflow:Assets written to: /home/penzo/workspace/MushDex/YOLO11/detection_training_final/fastervit/assets


In [ ]:
import pathlib
converter = tf.lite.TFLiteConverter.from_saved_model("fastervit/")
tflite_models_dir = pathlib.Path("tflite_models/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()
tflite_model_quant_file = tflite_models_dir/"fastervit_model_quant.tflite"
tflite_model_quant_file.write_bytes(tflite_quant_model)

In [ ]:
import pathlib

tflite_models_dir = pathlib.Path("/tf_lite_models/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]

In [ ]:
import onnxruntime as ort
import cv2
import numpy as np

# Carica il modello ONNX
ort_session = ort.InferenceSession("tl_model.onnx")

# Carica l'immagine
image = cv2.imread("4 (1).jpg")

# Ridimensiona l'immagine a 309x298
image_resized = cv2.resize(image, (298, 309))

# Converti l'immagine in float32 e normalizza i valori
input_data = image_resized.astype(np.float32) / 255.0

# Cambia l'ordine dei canali da HWC a CHW
#input_data = np.transpose(input_data, (2, 0, 1))

# Aggiungi una dimensione batch
input_data = np.expand_dims(input_data, axis=0)

# Ottieni il nome dell'input del modello
input_name = ort_session.get_inputs()[0].name

# Esegui l'inferenza
results = ort_session.run(None, {input_name: input_data})

# Ottieni le probabilità delle classi
probabilities = results[0][0]

# Ordina le probabilità e ottieni gli indici delle prime 5 classi
top_5_indices = np.argsort(probabilities)[-5:][::-1]

# Stampa i nomi delle classi con le probabilità più alte
for i in top_5_indices:
    print(f"Classe: {train_ds.class_names[i]}, Probabilità: {probabilities[i]:.4f}")

In [ ]:
# Ottieni il nome dell'input del modello
input_name = ort_session.get_inputs()[0].name

# Esegui l'inferenza
results = ort_session.run(None, {input_name: input_data})

# Stampa i risultati
print(results)

In [ ]:
import onnxruntime as ort
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

# Carica i nomi delle classi
class_names = np.load('classes.npy')

# Carica il modello ONNX
ort_session = ort.InferenceSession("tl_model.onnx")

# Estrai le etichette vere e le etichette predette
y_true = np.concatenate([y for x, y in validation_dataset], axis=0)
y_pred_probs = []

for x, y in validation_dataset:
    # Prepara i dati di input
    input_data = x.numpy().astype(np.float32)  # Converti in float32 invece di float16
    
    # Ottieni il nome dell'input del modello
    input_name = ort_session.get_inputs()[0].name
    
    # Esegui l'inferenza
    results = ort_session.run(None, {input_name: input_data})
    
    # Aggiungi i risultati alle probabilità predette
    y_pred_probs.append(results[0])

y_pred_probs = np.concatenate(y_pred_probs, axis=0)
y_pred_classes = np.argmax(y_pred_probs, axis=1)

# Matrice di confusione
conf_matrix = confusion_matrix(y_true, y_pred_classes)

# Trova le 10 classi con più problemi (errori)
errors = np.sum(conf_matrix, axis=1) - np.diag(conf_matrix)
top_10_problematic_classes = np.argsort(errors)[-10:]

# Visualizza la matrice di confusione per le 10 classi con più problemi
conf_matrix_top_10 = conf_matrix[top_10_problematic_classes][:, top_10_problematic_classes]
plt.figure(figsize=(12, 10))
sns.heatmap(conf_matrix_top_10, annot=True, cmap='Blues', fmt='g', xticklabels=[class_names[i] for i in top_10_problematic_classes], yticklabels=[class_names[i] for i in top_10_problematic_classes])
plt.title('Confusion Matrix for Top 10 Problematic Classes')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

# Report di classificazione
class_report = classification_report(y_true, y_pred_classes, target_names=class_names)
print('Classification Report')
print(class_report)

# Accuratezza
accuracy = accuracy_score(y_true, y_pred_classes)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Precisione
precision = precision_score(y_true, y_pred_classes, average='weighted')
print(f'Precision: {precision * 100:.2f}%')

# Richiamo
recall = recall_score(y_true, y_pred_classes, average='weighted')
print(f'Recall: {recall * 100:.2f}%')

# F1 Score
f1 = f1_score(y_true, y_pred_classes, average='weighted')
print(f'F1 Score: {f1 * 100:.2f}%')

In [ ]:
# Import some other useful libraries
import pandas as pd
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from prettytable import PrettyTable

In [ ]:
# Predict labels for the entire test set
predictions = ft_model.predict(val_ds, verbose=0)

# Display the shape of the predictions
print("Predictions Shape:", predictions.shape)

In [ ]:
# Extract the true labels and predicted labels
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

#y_true = np.concatenate([y for x, y in val_ds], axis=0)
#y_pred_probs = model.predict(val_ds)
y_pred_classes = np.argmax(y_pred_probs, axis=1)

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print('Confusion Matrix')
print(conf_matrix)

# Classification Report
class_report = classification_report(y_true, y_pred_classes)
print('Classification Report')
print(class_report)

# Accuracy
accuracy = accuracy_score(y_true, y_pred_classes)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Precision
precision = precision_score(y_true, y_pred_classes, average='weighted')
print(f'Precision: {precision * 100:.2f}%')

# Recall
recall = recall_score(y_true, y_pred_classes, average='weighted')
print(f'Recall: {recall * 100:.2f}%')

# F1 Score
f1 = f1_score(y_true, y_pred_classes, average='weighted')
print(f'F1 Score: {f1 * 100:.2f}%')

In [ ]:
from sklearn.metrics import roc_curve, auc

# Predicted probabilities for class 1 (unhealthy)
predicted_probabilities = predictions[:,1]
# True probabilities for class 1 (unhealthy)
true_probabilities = y_true[:]

# Compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(true_probabilities, predicted_probabilities)
roc_auc = auc(fpr, tpr)

# Calculate Sensitivity (True Positive Rate) and Specificity (1 - False Positive Rate)
sensitivity = tpr
specificity = 1 - fpr

# Calculate Youden's J statistic (tpr - fpr) for each threshold value
youden = sensitivity + specificity - 1

# Find the optimal threshold maximising the difference between tpr and fpr
optimal_threshold = thresholds[np.argmax(youden)]

# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], 'k--')  # Diagonal reference line

# Annotate the optimal threshold with a star marker
plt.scatter(fpr[np.argmax(youden)], tpr[np.argmax(youden)], marker='*',  color='black', s=100, label=f'Optimal Threshold = {optimal_threshold:.4f}', zorder=5)

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

print(f'Optimal Threshold: {optimal_threshold}')

In [ ]:
def predict_given_threshold(prediction_probabilities, threshold= 0.5):
  """
    Predict the class based on a threshold.
    For each element in the 'probabilities' array, the function assigns class 1 ('unhealthy') if the
    probability is greater than or equal to the threshold; otherwise, it assigns class 0 ('healthy').
    The result is an array of predicted classes reflecting the classification decisions for each
    corresponding probability in the input array.

    Parameters:
    - probabilities (array-like): Array or tensor of predicted probabilities of class 1.
    - threshold (float): The threshold value for classification.

    Returns:
    - array: Array of predicted classes (1 or 0).
  """
  return 1 * (prediction_probabilities[:,1] >= threshold)

In [ ]:
# Compute the new confusion matrix
confusion_matrix_new =  skl.metrics.confusion_matrix(y_true, predict_given_threshold(predictions, optimal_threshold))

# Compute classification metrics
#accuracy = accuracy_score(np.argmax(y_true, axis=-1), predict_given_threshold(predictions, optimal_threshold))
#precision = precision_score(np.argmax(y_true, axis=-1), predict_given_threshold(predictions, optimal_threshold), average='macro')
#recall = recall_score(np.argmax(y_true, axis=-1), predict_given_threshold(predictions, optimal_threshold), average='macro')
#f1 = f1_score(np.argmax(y_true, axis=-1), predict_given_threshold(predictions, optimal_threshold), average='macro')

# Accuracy
accuracy = accuracy_score(y_true, predict_given_threshold(predictions, optimal_threshold))
print(f'Accuracy: {accuracy * 100:.2f}%')

# Precision
precision = precision_score(y_true, predict_given_threshold(predictions, optimal_threshold), average='weighted')
print(f'Precision: {precision * 100:.2f}%')

# Recall
recall = recall_score(y_true, predict_given_threshold(predictions, optimal_threshold), average='weighted')
print(f'Recall: {recall * 100:.2f}%')

# F1 Score
f1 = f1_score(y_true, predict_given_threshold(predictions, optimal_threshold), average='weighted')
print(f'F1 Score: {f1 * 100:.2f}%')

"""# Display the computed metrics
print('Accuracy:', accuracy.round(4))
print('Precision:', precision.round(4))
print('Recall:', recall.round(4))
print('F1:', f1.round(4))"""

# Plot the confusion matrix
#plt.figure(figsize=(10, 8))
#sns.heatmap(confusion_matrix_new.T, cmap='Blues', annot= True, fmt='d')
#plt.xlabel('True labels')
#plt.ylabel('Predicted labels')
#plt.show()
# Plot confusion matrix
#plot_confusion_matrix(cm, class_names)
#plt.show()

In [ ]:
confusion_matrix_new

In [ ]:
conf_matrix

In [ ]:
optimal_threshold

In [ ]:
img_path = 'test3.jpg'
img = image.load_img(img_path, target_size=(224, 224))
# Preprocess the image: Convert it to a numpy array and scale it
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# If your model was trained on normalized images, you should normalize it here
img_array = img_array / 127.5 - 1  # Normalize pixel values to [-1, 1] range
#tl_model = tfk.models.load_model("new_model2/cp-0006.keras")
ft_model = tfk.models.load_model("new_FT2/cp-0003.keras")
# Predict the class
predictions = ft_model.predict(img_array)

In [ ]:
if (predictions[0][1]) > optimal_threshold:
    print("Da Raccogliere")
if (predictions[0][1]) <= optimal_threshold:
    print("Da Non Raccogliere")